# 한국고전종합DB XML
한국문집총간 XML : https://www.data.go.kr/data/3074298/fileData.do  
고전원문 XML : https://www.data.go.kr/data/15022432/fileData.do


In [ ]:
# !pip install gdown natsort

In [ ]:
import pandas as pd
import numpy as np
import lxml
import xml.etree.ElementTree as ET
from tqdm import tqdm
import numpy as np
import re
from natsort import natsorted
import glob

### 파일 목록 정리

In [ ]:
file_list = natsorted(glob.glob('./한국문집총간/*/*.xml'))

In [ ]:
print(len(file_list))
print(len(file_list)-1250) # 텍스트 존재 XML 파일 수

In [ ]:
# 텍스트 있는 xml 파일만 추출
file_list_text = [f for f in file_list if re.match('ITKC\_MO\_.+\_\d+\.xml',f.split('/')[-1])]
len(file_list_text)

In [ ]:
file_list_text[0].split('/')[-2] # 서명

In [ ]:
file_list_text[1].split('/')[-1] # 파일명

### 파싱 테스트

In [ ]:
tree = ET.parse(file_list_text[0])

In [ ]:
tree.find('.//레벨1').attrib['id']

In [ ]:
tree.find('.//레벨4').attrib['DCI']

In [ ]:
# 원문 id
tree.find('.//레벨4').attrib['id']

In [ ]:
# 연계항목(이미지, 번역, 교감표점) id
tree.findall('.//레벨4//연계항목')

In [ ]:
tree.findall('.//레벨4//연계항목')[0].attrib['type']

In [ ]:
tree.findall('.//레벨4//연계항목')[1].attrib['type']

In [ ]:
tree.findall('.//레벨4//연계항목')[1].attrib['연계시작']

In [ ]:
tree.find('.//레벨4//제목정보')

In [ ]:
tree.findall('.//레벨4//내용')

In [ ]:
tree.findall('.//레벨4//내용')[0].find('단락').text

In [ ]:
tree.findall('.//레벨4//내용')[0].findtext('단락')

In [ ]:
tree.findall('.//레벨4//내용')[0].attrib

In [ ]:
tree.findall('.//레벨4//내용')[0].tag

In [ ]:
for para in tree.iter('단락'):
  print(para.text)

In [ ]:
tree.findall('.//레벨4//내용')[0].find('단락').itertext()

In [ ]:
''.join(tree.findall('.//레벨4//내용')[0].find('단락').itertext())

In [ ]:
re.sub('\n','',''.join(tree.findall('.//레벨4//내용')[0].find('단락').itertext()))

### 실전 XML 파싱

In [ ]:
row_base_list = []
row_asso_list = []
for xml in tqdm(file_list_text):
    tree = ET.parse(xml)
    
    # 서지ID
    bib = tree.find('.//레벨1').attrib['id']
    
    # 서명
    book = xml.split('/')[-2] # 서명

    if tree.findall('.//레벨4')!=[]:
        # 기본정보
        lv4_list = tree.findall('.//레벨4')

        # 내용 부분
        lv4_content_list = tree.findall('.//레벨4//본문정보')

        # 연계항목
        # lv4_asso_list = tree.findall('.//레벨4//연계항목')
        
    else: #레벨3에 위치한 경우
        # print('lv3')
        # 기본정보
        lv4_list = tree.findall('.//레벨3')

        # 내용 부분
        lv4_content_list = tree.findall('.//레벨3//본문정보')

        # 연계항목
        # lv4_asso_list = tree.findall('.//레벨3//연계항목')


    # 자료ID
    id = [lv4.attrib['id'] for lv4 in lv4_list]

    # DCI_s
    dci = [lv4.attrib['DCI'] for lv4 in lv4_list]

    ## 내용
    content = [''.join(lv4_content.find('내용').itertext()).strip() for lv4_content in lv4_content_list]

    
    # ## 번역문ID
    # transID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='번역문']
    # ## 교감표점ID
    # kyoID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='교감표점']
    # ## 이미지ID
    # imageID = [(lv4_asso.attrib['연계시작'],lv4_asso.attrib['연계종료']) for lv4_asso in lv4_asso_list if lv4_asso.attrib['type']=='이미지']

    # 연계정보
    res_asso = [{'id':lv4.attrib['id'],'type':asso.attrib['type'], '연계시작':asso.attrib['연계시작'],'연계종료':asso.attrib['연계종료']} for lv4 in lv4_list for asso in lv4.findall('.//연계항목')]
        
    # dict로 합치기
    # res = {'bib':bib, 'book':book, 'id':id, 'dci':dci, 'content':content, 'transID':transID, 'kyoID':kyoID, 'imageID':imageID}
    res_base = {'bib':bib, 'book':book, 'id':id, 'dci':dci, 'content':content}

    row_base_list.append(res_base)
    row_asso_list.extend(res_asso)


In [ ]:
df = pd.DataFrame(row_base_list)
df

In [ ]:
# id가 비어있는 행 제외
print(df[df['id'].map(lambda x:len(x)==0)])
df = df[df['id'].map(lambda x:len(x)!=0)]

In [ ]:
# 중복 삭제
df = df[~df['book'].str.contains('\(1\)')].reset_index(drop=True)

In [ ]:
df_asso = pd.DataFrame(row_asso_list)
df_asso

In [ ]:
df_asso = df_asso.drop_duplicates().reset_index(drop=True)
df_asso

In [ ]:
len(set(df_asso[df_asso['type']=='번역문']['id'].tolist()))

In [ ]:
df_asso[df_asso['연계시작']!=df_asso['연계종료']]['type'].value_counts()

In [ ]:
df.to_pickle('한국문집총간_base.pickle')
df_asso.to_pickle('한국문집총간_asso.pickle')

In [ ]:
# 연계ID 비어있는 부분 채우기 (fillna)
# df['transID'] = df.apply(lambda x:[()]*len(x['content']) if (x['transID']==[]) else x['transID'],axis=1)
# df['kyoID'] = df.apply(lambda x:[()]*len(x['content']) if (x['kyoID']==[]) else x['kyoID'],axis=1)
# df['imageID'] = df.apply(lambda x:[()]*len(x['content']) if (x['imageID']==[]) else x['imageID'],axis=1)

In [ ]:
df_explode = df.explode(df.columns[2:].tolist(),ignore_index=True)
df_explode

In [ ]:
df_explode = df_explode.drop_duplicates(subset=['bib','id','dci','content']).reset_index(drop=True)
df_explode

In [ ]:
df_explode.to_pickle('한국문집총간_explode.pickle')